# Embed large text files using Batch Transform

If you have extensive text datasets that need embeddings using Jina's models, Amazon SageMaker's Batch Transform is a handy tool. Instead of processing text one-by-one, Batch Transform allows for bulk processing. Simply provide the path to your dataset in an S3 bucket and specify an output path. Once the transform job is completed, the embeddings will be uploaded to the designated S3 location.

In [ ]:
# !pip install --upgrade jina-sagemaker

In [1]:
from jina_sagemaker import Client
import boto3

region = boto3.Session().region_name

# Specify the role if needed
role = ""
role = 'arn:aws:iam::253352124568:role/service-role/AmazonSageMaker-ExecutionRole-20230527T104084'

# Specify the model name
model_name = "jina-embedding-l-en-v1"

# Mapping for Model Packages
model_package_map = {
    "us-east-1": f"arn:aws:sagemaker:us-east-1:253352124568:model-package/{model_name}",
}

# Specify the model you want to use
if region not in model_package_map.keys():
    raise Exception(f"Current boto3 session region {region} is not supported.")

model_package_arn = model_package_map[region]

## Create the batch transform job

#### Gotchas

- The input file must be a CSV file with no headers. 
- The first column in the input file should be the document ID and the second column should be the text to be embedded.
- The output file is a jsonlines file with extension (.out) with the document ID and the embedding.

In [2]:
client = Client(region_name=region)

# input_data_path_batch = "s3://sagemaker-us-east-1-253352124568/a.csv"
# output_data_path = "s3://sagemaker-us-east-1-253352124568/output/a"

input_data_path_batch = 'input.csv'
output_data_path = 'output'

client.create_transform_job(
    arn=model_package_arn,
    role=role,
    n_instances=1,
    instance_type="ml.g4dn.xlarge",
    input_path=input_data_path_batch,
    output_path=output_data_path,
)

Output path is output.


Using already existing model: jina-embedding-l-en-v1
INFO:sagemaker:Creating transform job with name: jina-embedding-l-en-v1-2023-10-05-07-14-39-487


...................................#033MMMMMMMMMMMMMMMMMMMMMMMMMMMMMM#033M#033M#033WWW#033MM#033MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM#033
#033MMMMMMMMMMMMMMMMMMMMMMMMMMMMM#033WNNNNNNNW#033M#033MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM#033
#033MMMMMMMMMMMMMMMMMMMMMMMMMMMM#033M#033NNNNNNNNN#033M#033MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM#033
#033MMMMMMMMMMMMMMMMMMMMMMMMMMMM#033M#033WNNNNNNNN#033M#033MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM#033
#033MMMMMMMMMMMMMMMMMMMMMMMMMMMMM#033M#033M#033WNNNW#033W#033M#033MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM#033
#033MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM#033MMM#033MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM#033
#033MMMMMMMMMMMM#033W#033xxxxxxxxx#033O#033MMMMM#033N#033xxxxxxxxx#0330#033MMMMM#033K#033ddddddx#033k#033K#033W#033MMMMMMMMMMM#033M#033X#033O#033xd#033d#033dx#033O#033N#033M#033MMM#033
#033MMMMMMMMMMMM#033X#033lllllllll#033d#033MMMMM#0330#033lllllllll#033x#033MMMMM#033O#033llllllllll#033o#0330#033M#033MMMMMM#033M#0330#033o#033lllllll

In [4]:
!ls

'Batch transform.ipynb'
'Inference with Jina Embedding Model.ipynb'
 input.csv
'Real time inference with Autoscaling.ipynb'


In [ ]:
client.download_s3_folder(
    client._sm_session.default_bucket(), output_data_path, output_data_path)